In [1]:
import os
import sys
import time
import yaml
import pandas as pd
import numpy as np

with open('../../config.local.yaml', 'r') as f:
    local_config = yaml.safe_load(f)

LOCAL_PATH = local_config['LOCAL_PATH']

sys.path.append(os.path.join(LOCAL_PATH, "src/python"))

from llm import get_response

In [2]:
meetings_df = pd.read_csv(os.path.join(LOCAL_PATH, "intermediate_data/cpc/meetings-manifest.csv"))
DATES = sorted(list(meetings_df['date']))

In [3]:
PROMPT = """
--- AGENDA ITEM ----

{}

--- MINUTES OF DISCUSSION ----

{}

--- PROMPT ----

I just gave you two documents related to a Los Angeles City Planning Commission (CPC) hearing.

The first document is the agenda item to be discussed, with requested actions.

The second document is the minutes of the discussion, the proposed motion by the CPC, the votes on the motion by the CPC members, and whether the motion ultimately passed.

Please return a response in the following format:

---- YOUR RESPONSE FORMAT ----
RELATED CASES:
<A comma separated list of relevant planning department case numbers>

SUMMARY OF AGENDA ITEM:
<A summary of the agenda item to be discussed>

SUMMARY OF CPC DELIBERATIONS:
<A summary of the deliberations of the CPC>

SUMMARY OF CPC MOTION:
<A summary of the motion voted on by the CPC>

VOTES FOR:
<A comma separated list of commission members who voted for the motion. If multiple motions were made, use only the last motion. If no one voted for, say "NONE". If no motion was made, say "N/A".>

VOTES AGAINST:
<A comma separated list of commission members who voted against the motion. If multiple motions were made, use only the last motion. If no one voted against, say "NONE". If no motion was made, say "N/A".>

VOTES ABSENT:
<A comma separated list of commission members who were absent or abstained for the vote. If multiple motions were made, use only the last motion. If no one was absent or abstained, say "NONE". If no motion was made, say "N/A".>

VOTE RESULT:
<Did the motion pass or fail? Your only options are MOTION PASSED, MOTION FAILED, N/A>

RESULT OF APPEAL:
<If the agenda item involved an appeal, say whether the appeal was granted or denied. Your only options are: APPEAL GRANTED, APPEAL GRANTED IN PART, APPEAL DENIED, NO APPEAL, DELIBERATIONS CONTINUED TO FUTURE DATE>

IMPLICATION FOR PROJECT:
<What is the implication of the vote for the original project proposal? Your only options are: APPROVED, APPROVED WITH MINOR CHANGES, APPROVED WITH MAJOR CHANGES, DENIED, DELIBERATIONS CONTINUED TO FUTURE DATE>
"""

In [4]:
t0 = time.time()
for date in DATES:
    year = date[0:4]
    print(date)
    PATH = os.path.join(LOCAL_PATH, f"intermediate_data/cpc/{year}/{date}")
    agenda_df = pd.read_pickle(os.path.join(PATH, 'agenda-items.pkl'))
    minutes_df = pd.read_pickle(os.path.join(PATH, 'minutes-items.pkl'))
    
    df = []
    for j, row2 in agenda_df.iterrows():
        item_no = row2['item_no']
        title = row2['title']
        is_casenum = row2['is_casenum']
        if is_casenum:
            print(f"{item_no}... ", end='')
            agenda_content = row2['content']
            minutes_content = minutes_df.loc[minutes_df['item_no']==item_no].iloc[0]['content']
            prompt = PROMPT.format(agenda_content, minutes_content)
            response, score = get_response(prompt)
            df.append({
                'year': year,
                'date': date,
                'item_no': item_no,
                'title': title,
                'prompt': prompt,
                'response': response,
                'score': score
            })
    print('')
    df = pd.DataFrame.from_dict(df)
    if len(df)>0:
        df.to_pickle(os.path.join(PATH, 'minutes-summaries.pkl'))
            
t1 = time.time()
print(f"Elapsed time: {(t1-t0)/60} minutes.")


2018-05-10
5a... 6... 7... 8... 9... 10... 
2018-05-23
5a... 6... 7... 8... 9... 10... 11... 
2018-06-14
5a... 6... 7... 8... 9... 10... 11... 
2018-07-12
7... 8... 9... 10... 11... 
2018-07-26
6... 7... 8... 
2018-08-09
5a... 6... 7... 8... 9... 10... 11... 12... 13... 
2018-08-23
5a... 6... 7... 
2018-09-13
5a... 5b... 6... 7... 8... 9... 10... 11... 12... 13... 
2018-09-27
5a... 6... 7... 8... 9... 10... 
2018-10-11
5a... 6... 7... 8... 9... 10... 11... 
2018-10-25
6... 7... 8... 9... 10... 
2018-11-08
5a... 6... 7... 8... 9... 10... 11... 
2018-11-29
6... 7... 8... 
2018-12-13
6... 7... 8... 
2018-12-20
6... 7... 
2019-01-10
6... 7... 8... 9... 
2019-01-24
5a... 5b... 6... 7... 8... 9... 10... 11... 
2019-02-14
6... 7... 8... 9... 10... 
2019-02-28
5a... 5b... 6... 7... 8... 
2019-03-14
5a... 5b... 6... 7... 8... 9... 10... 11... 12... 13... 
2019-03-28
5a... 5b... 5c... 6... 7... 8... 9... 10... 
2019-04-11
6... 7... 8... 9... 
2019-05-09
5a... 5b... 6... 7... 8... 9... 10... 11..